In [6]:
import numpy as np


#rule [axis, value, side]
# side = False: <=, side = True: >
class DetNode:

    @property
    def is_leaf(self):
        return self.density is not None
    
    @property
    def children(self):
        return self.left_child, self.right_child

    def __init__(self):
        self.density = None
        self.rules = np.empty((0,3))
        self.next_rule = 0
        self.neighbors_by_rules = []
        self.neighbors_by_id = {}
        self.left_child = None
        self.right_child = None
        self.id = ''

    def add_density(self, density):
        self.density = density

    def copy(self):
        new = DetNode()
        new.density = self.density
        new.rules = self.rules
        new.neighbors_by_id = self.neighbors_by_id
        new.neighbors_by_rules = self.neighbors_by_rules
        new.id = self.id
        return new

    def add_rule(self, new_rule, new_neighbor):
        new_rule_axis = new_rule[0]
        new_rule_side = new_rule[2]
        conflicting_rules_indices = np.where(np.logical_and(self.rules[:,0] == new_rule_axis, self.rules[:,2] == new_rule_side))[0]
        non_conflicting_rules_indices = list(set(range(len(self.rules))) - set(conflicting_rules_indices))

        updated_rules = np.empty((0,3))
        updated_neighbors_by_rules = []
        updated_neighbors_by_id = {}
        counter = 0
        for index in non_conflicting_rules_indices:
            updated_rules = np.vstack((updated_rules, self.rules[index]))
            neighbors = self.neighbors_by_rules[index]
            updated_neighbors_by_rules.append(neighbors)
            for neighbor in neighbors:
                updated_neighbors_by_id[neighbor.id] = (neighbor, counter)
            counter += 1
        self.rules = updated_rules
        self.neighbors_by_rules = updated_neighbors_by_rules
        self.neighbors_by_id = updated_neighbors_by_id
        conflict2_rules_indices = np.where(np.logical_and(self.rules[:,0] == new_rule_axis, self.rules[:,2] != new_rule_side))[0]
        for i in conflict2_rules_indices:
            for neighbor in self.neighbors_by_rules[i]:
                neighbor.fix_conflict(self)
        conflict3_rules_indices = np.where(self.rules[:,0] != new_rule_axis)[0]
        for i in conflict3_rules_indices:
            for neighbor in self.neighbors_by_rules[i]:
                neighbor.fix_conflict(self)
        self.neighbors_by_id[new_neighbor.id] = (new_neighbor, len(self.rules))
        self.neighbors_by_rules.append([new_neighbor])
        self.rules = np.vstack((self.rules, new_rule))

    def fix_conflict(self, new_neighbor):
        new_id = new_neighbor.id
        old_id = new_neighbor.id[:-1]
        if old_id in self.neighbors_by_id:
            old_neighbor, old_neighbor_rule_index = self.neighbors_by_id[old_id]
            del(self.neighbors_by_id[old_id])
            self.neighbors_by_rules[old_neighbor_rule_index].remove(old_neighbor)
        elif old_id + '0' in self.neighbors_by_id:
            other_neighbor, old_neighbor_rule_index = self.neighbors_by_id[old_id + '0']
        elif old_id + '1' in self.neighbors_by_id:
            other_neighbor, old_neighbor_rule_index = self.neighbors_by_id[old_id + '1']
        self.neighbors_by_id[new_id] = (new_neighbor, old_neighbor_rule_index)
        self.neighbors_by_rules[old_neighbor_rule_index].append(new_neighbor)


    def split(self, axis, value):
        left_child = self.copy()
        right_child = self.copy()
        left_rule = [axis, value, False]
        right_rule = [axis, value, True]
        left_child.id += '0'
        right_child.id += '1'
        left_child.add_rule(left_rule, right_child)
        right_child.add_rule(right_rule, left_child)
        self.left_child = left_child
        self.right_child = right_child
        return left_child, right_child

In [7]:
root = DetNode()
n1, n2 = root.split(0,0.5)
n5, n6 = n1.split(0,0.25)
n4, n3 = n5.split(1,0.5)

In [8]:
print n1.id
print n2.id
print n3.id
print n4.id
print n5.id
print n6.id

0
1
001
000
00
01


In [30]:


def create_tree(data):
    root = DetNode()
    split_node(data, root, None)
    return root


def split_node(data, node, line_data):
    if line_data is not None and 'density' in line_data:
        node.density = line_data['density']
    if len(data) > 1:
        right_child = data[0]
        counter = 0
        for datum in data[1:]:
            axis = right_child['axis']
            counter += 1
            if datum['axis'] == axis and datum['value'] == right_child['value']:
                left_child = datum
                break

        right_data = data[1:counter]
        left_data = data[(counter+1):]
        left_node, right_node = node.split(axis, value)
        split_node(left_data, left_node, left_child)
        split_node(right_data, right_node, right_child)


file = open('sub_tree.txt', 'r')
lines =  [line.strip(' \t\n\r|') for line in file.readlines()]
data = []
for line in lines:
    line_data = line.split(' ')
    axis = int(line_data[1])
    side = line_data[2] == '>'
    value = float(line_data[3].replace(':', ''))
    line_data_hash = {'axis': axis, 'side': side, 'value': value}
    density = None
    if len(line_data) > 4:
        density = float(line_data[4][5:])
        line_data_hash['density'] = density
    data.append(line_data_hash)
root = create_tree(data)

In [33]:
n1, n2 = root.children
n3, n4 = n1.children
n5, n6 = n4.children
print n1.rules
print n2.rules
print n3.rules
print n4.rules
print n5.rules
print n6.rules

[[    0.    7461.65     0.  ]]
[[  0.00000000e+00   7.46165000e+03   1.00000000e+00]]
[[  0.00000000e+00   7.46165000e+03   0.00000000e+00]
 [  1.00000000e+00   7.46165000e+03   0.00000000e+00]]
[[  0.00000000e+00   7.46165000e+03   0.00000000e+00]
 [  1.00000000e+00   7.46165000e+03   1.00000000e+00]]
[[  0.00000000e+00   7.46165000e+03   0.00000000e+00]
 [  1.00000000e+00   7.46165000e+03   1.00000000e+00]
 [  2.00000000e+00   7.46165000e+03   0.00000000e+00]]
[[  0.00000000e+00   7.46165000e+03   0.00000000e+00]
 [  1.00000000e+00   7.46165000e+03   1.00000000e+00]
 [  2.00000000e+00   7.46165000e+03   1.00000000e+00]]


In [25]:
root = DetNode()
root.split(0,0.1)
n1, n2 = root.children
print n1

<__main__.DetNode instance at 0x7fe7ec6e0638>


In [78]:
'a' in d

True

In [35]:
import numpy as np
from sklearn.decomposition import PCA
original = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=2)
transformed = pca.fit_transform(X)

In [49]:
import scipy.spatial.distance as distance
original_dist = distance.pdist(original, 'cityblock')
transformed_dist = distance.pdist(transformed, 'cityblock')
print original_dist - transformed_dist < 1e-15

[ True False False False False False False False False False False False
  True False False]


In [43]:
print original_dist

[ 1.          2.23606798  2.82842712  3.60555128  5.          1.41421356
  3.60555128  4.47213595  5.83095189  5.          5.83095189  7.21110255
  1.          2.23606798  1.41421356]


In [44]:
print transformed_dist

[ 1.          2.23606798  2.82842712  3.60555128  5.          1.41421356
  3.60555128  4.47213595  5.83095189  5.          5.83095189  7.21110255
  1.          2.23606798  1.41421356]
